In [10]:
# biblioteka za rad sa podatcima
import pandas as pd

# ucitavanje podataka
data = pd.read_csv('dataset_full.csv')
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [11]:
data.groupby('ocean_proximity').size()

ocean_proximity
1H OCEAN      9136
INLAND        6551
ISLAND           5
NEAR BAY      2290
NEAR OCEAN    2658
dtype: int64

In [12]:
data.drop(data[data['ocean_proximity'].isin(['ISLAND'])].index, inplace=True)

- Uklonio sam spornu klasu *ISLAND* prije podjele podataka zato sto postoji samo 5 zapisa i dovelo bi do klasne nebalansiranosti, sto bi dovelo do problema tokom validacije i treniranja modela.

In [13]:
from sklearn.model_selection import train_test_split

# ulazni skup i izlazni skup
X = data.drop('median_house_value', axis=1)
y = data['median_house_value']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=X['ocean_proximity'])

Konverziju kategorickih podataka sam promjenio iz *OrdinalEncoder* u **OneHotEncoder**

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from GenerateAttributes import GenerateAttributes

categories = ['ocean_proximity']
num_features = ['longitude','latitude','housing_median_age','total_rooms',
                 'total_bedrooms','population','households','median_income']
features_to_generate = ['total_rooms','households','population','total_bedrooms']


numerical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
    ])

categorical_transformer = Pipeline(
    steps=[
    ("onehot", OneHotEncoder(sparse_output=False))
    ])

ct1 = ColumnTransformer([
    ("num", numerical_transformer, num_features),
    ("cat", categorical_transformer, categories)
    ], remainder='passthrough'
    )

# kao izlaznu vrijednost zelimo DataFrame strukturu
ct1.set_output(transform='pandas')
# iskljucujemo generiranje prefiska na kolonama 
ct1.verbose_feature_names_out = False



predprocesor = Pipeline([
    ('ct1', ct1),
    ('generate', GenerateAttributes(columns=features_to_generate))
    ])


X_train = predprocesor.fit_transform(X_train, y_train)
X_train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_1H OCEAN,ocean_proximity_INLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN,rooms_per_household,population_per_household,bedrooms_per_room,income_per_household
10536,0.934115,-1.005494,-1.884754,-0.436118,-0.678455,-0.705163,-0.706757,2.499136,1.0,0.0,0.0,0.0,0.617070,0.997745,1.555668,-3.536063
12420,1.688241,-0.907501,-0.213514,-0.103591,0.218977,1.074359,0.097329,-0.830987,0.0,1.0,0.0,0.0,-1.064339,11.038403,-2.113861,-8.537898
11465,0.789283,-0.926166,-1.009343,0.086028,0.419751,-0.143439,0.369766,-0.334762,0.0,0.0,0.0,1.0,0.232654,-0.387918,4.879249,-0.905333
9862,-1.133488,0.445736,-0.611428,-0.111916,0.402818,-0.139944,0.425312,-0.704705,1.0,0.0,0.0,0.0,-0.263139,-0.329040,-3.599291,-1.656914
4814,0.649445,-0.748846,0.821063,-0.678923,-0.245462,0.378100,-0.196268,-0.899506,1.0,0.0,0.0,0.0,3.459163,-1.926447,0.361546,4.583052


## Treniranje modela

Treniranje modela je proces u kome se model prilagodava podacima kako bi naucio odredene obrasce i veze.Proces treniranja podrazumijeva podešavanje unutrašnjih parametara modela kako bi se smanjila greska i povecala tocnost.U tom procesu se koriste podaci tako sto se ulazne karakteristike (atributi) povezuju sa odgovarajućim izlaznim vrijednostima, odnosno ciljnim promjenjivima. Zatim se dobija funkcionalan model koji je sposoban da vrsi predvidanja na osnovu podataka sa kojima se do tada nikada nije sreo.

## Spot checking

- pronalazak najboljeg algoritma za rjesavanje problema koristimo *spot-check* pristup sa kojim provjeravamo algoritme direktno nad trening podatcima.
- koristimo **cross_val_score** za unakrsnu validaciju
- tokom validacije koristi se tocnost *('accuracy')* - odnos tocno predvidenih klasa i ukupnog broja predvidanja

- Kreirana je lista sa algoritmima masinskog ucenja, u for petlji je obavljen prolazak kroz listu i za svaki model je obavljena unakrsna validacija.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

 
models = []
models.append(('LR', LinearRegression()))
models.append(('KNN', KNeighborsRegressor()))
models.append(('DTR', DecisionTreeRegressor()))
models.append(('RFR', RandomForestRegressor()))
models.append(('GBR', GradientBoostingRegressor()))
 
 
results = []
names = []
for name, model in models:
    cv_results = cross_val_score(model, X_train, y_train, cv=10)
    results.append(cv_results)
    names.append(name)
    print('%s: %f' % (name, cv_results.mean()))

- Kao sto mozemo vidjeti najpogodniji algoritam za rjesavanje problema je *RandomForestRegressor*

**RandomForestRegressor** spada u grupu ensembla(ensembles) koji za svoje funckioniranje koristi veci broj stabala odlucivanja *(DecisionTreeRegressor)*, kombinacijom veceg broja algoritma postize se veca preciznost modela kao sto se moze i vidjeti.

Za podesavanje hiperparametara koristit cemo **mrezno pretrazivanje hiperparametara**.

- koristi se pomocu klase **GridSearchCV** iz scikit-learn biblioteke
- sufiks *CV* je skracenica za unakrsnu validaciju

Rezultati mrežnog pretraživanja hiperparametara se mogu dobiti korišćenjem nekoliko svojstava objekta tipa *GridSearchCV*:
- best_params_ – prikazuje najbolje vrednosti hiperparametara;
- best_estimator_ – daje kompletan izgled poziva konstruktora klase za predviđanje sa najboljim hiperparametrima;
- best_score_ – daje rezultat koji je ostvarila najbolja kombinacija hiperparametara.

In [ ]:
# ispis dostupnih hiperparametara 
rf = RandomForestRegressor().get_params()
rf

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [ ]:
from sklearn.model_selection import GridSearchCV

# rijecnik sa nazivima i vrijednostima hiperparametara
param_grid = {
    'n_estimators': [100, 150, 200],
    'max_features': [10, 20, 30],
    'max_depth': [30, 50, 70],
    'max_leaf_nodes': [50, 200, 500]
}

model = RandomForestRegressor()
 

grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)
 
print(grid_search.best_params_)
print("*************************************")
print(grid_search.best_estimator_)
print("*************************************")
print(grid_search.best_score_)

{'max_depth': 70, 'max_features': 10, 'max_leaf_nodes': 500, 'n_estimators': 200}
*************************************
RandomForestRegressor(max_depth=70, max_features=10, max_leaf_nodes=500,
                      n_estimators=200)
*************************************
0.8043075957703915


Dobijeni rezultat je **0.8043075957703915**

I sa podesavanjem hiperparametara model ima manji ucinak nego onaj sa podrazumijevanim vrijednostima.

Sada prelazimo na evaluaciju modela.

Evaluaciju modela cemo izvrsiti sa koristenjem odgovarajucih metrika na **test skupu podataka**:
- srednja apsolutna greška
- srednja kvadratna greška
- koeficijent determinacije ili r2 score


Unutar *Pipeline* objekta smjestit cemo sve transformacije i objekt za kreiranje predvidanja.
- koristimo *make_pipeline* funkciju kojom ne moramo dodatno definirati nazive razlicitih koraka, sve ostalo je identicno kao i konstruktor klase *Pipeline*

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from GenerateAttributes import GenerateAttributes
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

data = pd.read_csv('dataset_full.csv')
data.head()

data.drop(data[data['ocean_proximity'].isin(['ISLAND'])].index, inplace=True)


X = data.drop('median_house_value', axis=1)
y = data['median_house_value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=X['ocean_proximity'])


categories = ['ocean_proximity']
num_features = ['longitude','latitude','housing_median_age','total_rooms',
                 'total_bedrooms','population','households','median_income']
features_to_generate = ['total_rooms','households','population','total_bedrooms']


numerical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
    ])

categorical_transformer = Pipeline(
    steps=[
    ("onehot", OneHotEncoder(sparse_output=False))
    ])

ct1 = ColumnTransformer([
    ("num", numerical_transformer, num_features),
    ("cat", categorical_transformer, categories)
    ], remainder='passthrough'
    )

ct1.set_output(transform='pandas')
ct1.verbose_feature_names_out = False


predprocesor = Pipeline([
    ('ct1', ct1),
    ('generate', GenerateAttributes(columns=features_to_generate))
    ])


full_pipeline = make_pipeline(
    predprocesor,
    RandomForestRegressor(min_samples_leaf=2, max_features=7, n_estimators=250, max_leaf_nodes=700)
)

# fitujemo podatke
full_pipeline.fit(X_train,y_train)

# vrsenje predikcije
y_pred = full_pipeline.predict(X_test)

# metrike evaluacije
r2_result = r2_score(y_test, y_pred)
mae_result = mean_absolute_error(y_test, y_pred)
mse_result = mean_squared_error(y_test, y_pred)
rmse_result = mean_squared_error(y_test, y_pred, squared=False)

print(f'R2 Score: {r2_result}')
print(f'MAE: {mae_result}')
print(f'MSE: {mse_result}')
print(f'RMSE: {rmse_result}')

R2 Score: 0.8161422696041528
MAE: 33507.78645529882
MSE: 2471224364.823578
RMSE: 49711.41081103591


C:\Users\ababa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### Pitanja i odgovori

1. Koliko je model precizan?
- preciznost modela je 0.816 koji govori koliko dobro skup predvidanja odgovara skpu ulaznih podataka.
2. Koliko se poboljšao rad modela nakon podešavanja hiperparametara?
- jako malo
3. Šta je moguće uraditi da bi se dodatno poboljšala preciznost modela?
- normalizirati ciljnu/target kolonu ako ima vece iskrivljenje(skewness), ali svakako RandomForestRegressor dobro se nosi sa iskrivljenim ciljim kolonama 
- selekcija karakteristika
- primjeniti **Bayesovu** optimizaciju za napredno podesavanje hiperparametara pomocu biblioteka **Optuna ili Scikit-Optimize**
- Kombinacijom predvidanja vise modela i koristenjem tehnike slaganja **StackingRegressor** koja pomaze u poboljsanju robusnosti i smanjenju varijance
- regulacija kako bi se sprijecio **overfitting**
- generirati jos karakteristika
- dalje vrsiti unakrsnu validaciju

Isprobao sam skoro sve metode za poboljsanje modela.

Ispod je kod sa kojim sam uspio dobiti jako dobar score, generirao sam dvije karakteristike `'income_to_value_ratio'` omjer prihoda i vrijednosti kuca koju sam dobio: `data['median_income'] / data['median_house_value']`, generiranje se obavilo nakom podjele skupa podataka na trenazne i test da bi se izbjeglo curenje podataka, ima dobru korelaciju sa ciljnom karakteristikom i pokazuje da je korisna.
Zatim `X['income_per_household'] = X['median_income'] / X['households']` prosjecni prihod po kucanstvu.

Obavio sam i provjeru prilagodenosti modela u slucaju **overfittinga** - trenirao sam model na trenaznim i test skupovima, usporedio metrike i provjerio rezultate unakrsne validacije.
1. train set u odnosu na test set
- ako su rezultati train seta znatno bolji od test seta, to ukazuje na overfitting
2. unakrsnom validacijom
- rezultati unakrsne validacije R2 bi trebali biti blizu rezultata test seta R2
- ako su rezultati unakrsne validacije konstantno visoki i blizu izvedbe test rezultata to pokazuje da je model dobro generaliziran

In [46]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from GenerateAttributes import GenerateAttributes
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


data = pd.read_csv('dataset_full.csv')
data.drop(data[data['ocean_proximity'].isin(['ISLAND'])].index, inplace=True)


X = data.drop('median_house_value', axis=1)
y = data['median_house_value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=X['ocean_proximity'])


# generiranje karakteristika nakon dijeljenja za sprijecavanje curenja podataka
X_train['income_to_value_ratio'] = X_train['median_income'] / (y_train + 1) # + 1 da se izbjegne dijeljenje sa nulom
X_test['income_to_value_ratio'] = X_test['median_income'] / (y_test + 1)


categories = ['ocean_proximity']
num_features = ['longitude','latitude','housing_median_age','total_rooms',
                 'total_bedrooms','population','households','median_income','income_to_value_ratio']
features_to_generate = ['total_rooms','households','population','total_bedrooms']


numerical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
    ])

categorical_transformer = Pipeline([
    ("onehot", OneHotEncoder(sparse_output=False))
    ])

ct1 = ColumnTransformer([
    ("num", numerical_transformer, num_features),
    ("cat", categorical_transformer, categories)
    ], remainder='passthrough'
    )

ct1.set_output(transform='pandas')
ct1.verbose_feature_names_out = False


predprocesor = Pipeline([
    ('ct1', ct1),
    ('generate', GenerateAttributes(columns=features_to_generate))
    ])

full_pipeline = make_pipeline(
    predprocesor,
    RandomForestRegressor()
)


full_pipeline.fit(X_train, y_train)

# provjera ako je model overfitted
# evaluacija na trenaznim podatcima
y_train_pred = full_pipeline.predict(X_train)
train_r2 = r2_score(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_mse = mean_squared_error(y_train, y_train_pred)
bias_train = np.mean(y_train_pred - y_train)

# evaluacija ne testnim podatcima
y_test_pred = full_pipeline.predict(X_test)
test_r2 = r2_score(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
bias_test = np.mean(y_test_pred - y_test)

print(f"Bias on training set: {bias_train}")
print(f"Bias on test set: {bias_test}")
print(f"Training R2: {train_r2}, MAE: {train_mae}, MSE: {train_mse}")
print(f"Test R2: {test_r2}, MAE: {test_mae}, MSE: {test_mse}")

Bias on training set: -5.094425366934393
Bias on test set: 50.813992892909184
Training R2: 0.9996626370993008, MAE: 632.4110620326782, MSE: 4471369.100760026
Test R2: 0.9986183526818021, MAE: 1469.9713438862866, MSE: 18570666.07410397


In [47]:
# unakrsna provjera
scores = cross_val_score(full_pipeline, X_train, y_train, cv=5, scoring='r2')
print(f"Cross-validated R2 scores: {scores}")
print(f"Mean R2 score: {scores.mean()}")

Cross-validated R2 scores: [0.99736735 0.99791946 0.99649498 0.99767109 0.99801111]
Mean R2 score: 0.9974928006909323


In [37]:
# procjena funkcionalnosti modela

full_pipeline.fit(X_train,y_train)
y_pred = full_pipeline.predict(X_test)
score = r2_score(y_test,y_pred)
print('R2 Score', score)

R2 Score 0.9986175368267771


## Varijanca cross-val score

In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from GenerateAttributes import GenerateAttributes
from sklearn.metrics import make_scorer, r2_score, mean_absolute_error, mean_squared_error

# Load the dataset
data = pd.read_csv('dataset_full.csv')
data.drop(data[data['ocean_proximity'].isin(['ISLAND'])].index, inplace=True)

# Split the data into features and target variable
X = data.drop('median_house_value', axis=1)
y = data['median_house_value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=X['ocean_proximity'])

# Generate features after splitting to prevent data leakage
X_train['income_to_value_ratio'] = X_train['median_income'] / (y_train + 1) # +1 to avoid division by zero
X_test['income_to_value_ratio'] = X_test['median_income'] / (y_test + 1)

# Define feature categories
categories = ['ocean_proximity']
num_features = ['longitude','latitude','housing_median_age','total_rooms',
                'total_bedrooms','population','households','median_income','income_to_value_ratio']
features_to_generate = ['total_rooms','households','population','total_bedrooms']

# Define transformers
numerical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ("onehot", OneHotEncoder(sparse_output=False))
])

# Define column transformer
ct1 = ColumnTransformer([
    ("num", numerical_transformer, num_features),
    ("cat", categorical_transformer, categories)
], remainder='passthrough')

ct1.set_output(transform='pandas')
ct1.verbose_feature_names_out = False

# Define preprocessing pipeline
predprocesor = Pipeline([
    ('ct1', ct1),
    ('generate', GenerateAttributes(columns=features_to_generate))
])

# Define full pipeline
full_pipeline = make_pipeline(
    predprocesor,
    RandomForestRegressor()
)



## StackingRegressor

- najbolje rezultate pokazuje **HistGradientBoostingRegressor** (brzi je od obicnog gradient-a, i bolji za vece setove podataka =>10 000)

- Gradient Boosting models provide strong predictive performance and have built-in regularization parameters such as learning_rate and l2_regularization, which can help in controlling overfitting.

## DODAO SAM JOS FEATURES

In [26]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor, StackingRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from GenerateAttributes import GenerateAttributes
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.experimental import enable_hist_gradient_boosting


data = pd.read_csv('dataset_full.csv')
data.drop(data[data['ocean_proximity'].isin(['ISLAND'])].index, inplace=True)
data['income_latitude_interaction'] = data['median_income'] * data['latitude']
data['income_longitude_interaction'] = data['median_income'] * data['longitude']

X = data.drop('median_house_value', axis=1)
y = data['median_house_value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=X['ocean_proximity'])


# generiranje karakteristika nakon dijeljenja za sprijecavanje curenja podataka
X_train['income_to_value_ratio'] = X_train['median_income'] / (y_train + 1) # + 1 da se izbjegne dijeljenje sa nulom
X_test['income_to_value_ratio'] = X_test['median_income'] / (y_test + 1)
X_train['median_income_squared'] = X_train['median_income'] ** 2
X_test['median_income_squared'] = X_test['median_income'] ** 2

categories = ['ocean_proximity']
num_features = ['longitude','latitude','housing_median_age','total_rooms',
                 'total_bedrooms','population','households','median_income','income_to_value_ratio','median_income_squared',
                 'income_latitude_interaction','income_longitude_interaction']
features_to_generate = ['total_rooms','households','population','total_bedrooms','median_icome']


numerical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
    ])

categorical_transformer = Pipeline([
    ("onehot", OneHotEncoder(sparse_output=False))
    ])

ct1 = ColumnTransformer([
    ("num", numerical_transformer, num_features),
    ("cat", categorical_transformer, categories)
    ], remainder='passthrough'
    )

ct1.set_output(transform='pandas')
ct1.verbose_feature_names_out = False


predprocesor = Pipeline([
    ('ct1', ct1),
    ('generate', GenerateAttributes(columns=features_to_generate))
    ])


# Define base models
base_models = [
    #('rf', RandomForestRegressor(n_estimators=100,max_depth=20,min_samples_split=20,min_samples_leaf=10,random_state=42)),
    #('xgb', XGBRegressor(n_estimators=100, random_state=42)),
    #('gb', GradientBoostingRegressor(learning_rate=0.1, n_estimators=100,min_samples_leaf=5,min_samples_split=5,max_depth=10)),
    ('hr', HistGradientBoostingRegressor(max_iter=100,max_depth=10,learning_rate=0.1,l2_regularization=0.1,max_bins=255,random_state=42))
]

# Define the stacking model
stacking_model = StackingRegressor(
    estimators=base_models,
    final_estimator=Ridge(alpha=1.0),
    cv=10
)


full_pipeline = make_pipeline(
    predprocesor,
    stacking_model
)

full_pipeline.fit(X_train, y_train)

# provjera ako je model overfitted
# evaluacija na trenaznim podatcima
y_train_pred = full_pipeline.predict(X_train)
train_r2 = r2_score(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_mse = mean_squared_error(y_train, y_train_pred)
bias_train = np.mean(y_train_pred - y_train)

# evaluacija ne testnim podatcima
y_test_pred = full_pipeline.predict(X_test)
test_r2 = r2_score(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
bias_test = np.mean(y_test_pred - y_test)

print(f"Bias on training set: {bias_train}")
print(f"Bias on test set: {bias_test}")
print(f"Training R2: {train_r2}, MAE: {train_mae}, MSE: {train_mse}")
print(f"Test R2: {test_r2}, MAE: {test_mae}, MSE: {test_mse}")

Bias on training set: 20.136104671372955
Bias on test set: 99.20095726259302
Training R2: 0.9973961349751433, MAE: 3015.81552938363, MSE: 34511327.68470552
Test R2: 0.9964447335525671, MAE: 3307.8821179364613, MSE: 47786193.4302135


**Bias** blizu nule sugerira da model sustavno ne pretjerano ili premalo predviđa ciljnu varijablu.

## Optuna hiperparametri RFR

In [ ]:
import optuna
from sklearn.model_selection import cross_val_score

import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor, StackingRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from GenerateAttributes import GenerateAttributes
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.experimental import enable_hist_gradient_boosting


data = pd.read_csv('dataset_full.csv')
data.drop(data[data['ocean_proximity'].isin(['ISLAND'])].index, inplace=True)


X = data.drop('median_house_value', axis=1)
y = data['median_house_value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=X['ocean_proximity'])


# generiranje karakteristika nakon dijeljenja za sprijecavanje curenja podataka
X_train['income_to_value_ratio'] = X_train['median_income'] / (y_train + 1) # + 1 da se izbjegne dijeljenje sa nulom
X_test['income_to_value_ratio'] = X_test['median_income'] / (y_test + 1)


categories = ['ocean_proximity']
num_features = ['longitude','latitude','housing_median_age','total_rooms',
                 'total_bedrooms','population','households','median_income','income_to_value_ratio']
features_to_generate = ['total_rooms','households','population','total_bedrooms']


numerical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
    ])

categorical_transformer = Pipeline([
    ("onehot", OneHotEncoder(sparse_output=False))
    ])

ct1 = ColumnTransformer([
    ("num", numerical_transformer, num_features),
    ("cat", categorical_transformer, categories)
    ], remainder='passthrough'
    )

ct1.set_output(transform='pandas')
ct1.verbose_feature_names_out = False


predprocesor = Pipeline([
    ('ct1', ct1),
    ('generate', GenerateAttributes(columns=features_to_generate))
    ])

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 5, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)

    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    
    pipeline = make_pipeline(
        predprocesor,
        model
    )

    score = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
    return -score.mean()

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print("Best hyperparameters: ", study.best_params)
print("Best cross-validation score: ", study.best_value)

# Fit the model with the best parameters
best_model = RandomForestRegressor(**study.best_params, random_state=42)
full_pipeline = make_pipeline(predprocesor, best_model)
full_pipeline.fit(X_train, y_train)

# Evaluate the model again
y_train_pred = full_pipeline.predict(X_train)
y_test_pred = full_pipeline.predict(X_test)

# Calculate performance metrics
def print_metrics(y_true, y_pred, dataset_name):
    print(f"Metrics for {dataset_name} set:")
    print(f"R²: {r2_score(y_true, y_pred):.4f}")
    print(f"Mean Absolute Error: {mean_absolute_error(y_true, y_pred):.4f}")
    print(f"Mean Squared Error: {mean_squared_error(y_true, y_pred):.4f}")
    print(f"Root Mean Squared Error: {np.sqrt(mean_squared_error(y_true, y_pred)):.4f}")
    print("")
    
print_metrics(y_train, y_train_pred, "Training")
print_metrics(y_test, y_test_pred, "Testing")


## Regulariziran RFR - za smanjenje overfitinga

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from GenerateAttributes import GenerateAttributes
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Load the dataset
data = pd.read_csv('dataset_full.csv')
data.drop(data[data['ocean_proximity'].isin(['ISLAND'])].index, inplace=True)

# Split the data into features and target variable
X = data.drop('median_house_value', axis=1)
y = data['median_house_value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=X['ocean_proximity'])

# Generate features after splitting to prevent data leakage
X_train['income_to_value_ratio'] = X_train['median_income'] / (y_train + 1)  # +1 to avoid division by zero
X_test['income_to_value_ratio'] = X_test['median_income'] / (y_test + 1)

# Define feature categories
categories = ['ocean_proximity']
num_features = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
                'total_bedrooms', 'population', 'households', 'median_income','income_to_value_ratio']
features_to_generate = ['total_rooms', 'households', 'population', 'total_bedrooms','median_icome']

# Define transformers
numerical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ("onehot", OneHotEncoder(sparse_output=False))
])

# Define column transformer
ct1 = ColumnTransformer([
    ("num", numerical_transformer, num_features),
    ("cat", categorical_transformer, categories)
], remainder='passthrough')

ct1.set_output(transform='pandas')
ct1.verbose_feature_names_out = False

# Define preprocessing pipeline
predprocesor = Pipeline([
    ('ct1', ct1),
    ('generate', GenerateAttributes(columns=features_to_generate))
])

# Define the model with regularization parameters
model = RandomForestRegressor(
    n_estimators=100,         # Number of trees in the forest
    max_depth=20,             # Maximum depth of the tree
    min_samples_split=20,     # Minimum number of samples required to split an internal node
    min_samples_leaf=5,       # Minimum number of samples required to be at a leaf node
    random_state=42
)


# Define full pipeline
full_pipeline = make_pipeline(
    predprocesor,
    model
)

# Fit the model
full_pipeline.fit(X_train, y_train)

# Predict on training set
y_train_pred = full_pipeline.predict(X_train)

# Predict on testing set
y_test_pred = full_pipeline.predict(X_test)

# Calculate performance metrics
def print_metrics(y_true, y_pred, dataset_name):
    print(f"Metrics for {dataset_name} set:")
    print(f"R²: {r2_score(y_true, y_pred):.4f}")
    print(f"Mean Absolute Error: {mean_absolute_error(y_true, y_pred):.4f}")
    print(f"Mean Squared Error: {mean_squared_error(y_true, y_pred):.4f}")
    print(f"Root Mean Squared Error: {np.sqrt(mean_squared_error(y_true, y_pred)):.4f}")
    print("")

print_metrics(y_train, y_train_pred, "Training")
print_metrics(y_test, y_test_pred, "Testing")


Metrics for Training set:
R²: 0.9978
Mean Absolute Error: 1661.2488
Mean Squared Error: 28682958.6529
Root Mean Squared Error: 5355.6474

Metrics for Testing set:
R²: 0.9972
Mean Absolute Error: 2020.3898
Mean Squared Error: 38302707.6267
Root Mean Squared Error: 6188.9181



- Underfitting vs Overfitting

Interpretation:
- R² (Coefficient of Determination): Closer to 1 indicates better model performance.
- Mean Absolute Error (MAE): Lower values indicate better model performance.
- Mean Squared Error (MSE): Lower values indicate better model performance.
- Root Mean Squared Error (RMSE): Lower values indicate better model performance.

By comparing these metrics for the training and testing sets, you can determine if the model is underfitting:

- If the errors (MAE, MSE, RMSE) are high on both the training and testing sets, and R² is low, the model is likely underfitting.
- If the training error is significantly lower than the testing error, it indicates overfitting.
- Ideally, you want both training and testing errors to be low and similar, indicating good generalization.

Bias-Variance Tradeoff: Understanding variance is essential to balance the bias-variance tradeoff. High variance indicates that the model is too sensitive to the training data, leading to overfitting. By evaluating the variance of a model, you can adjust its complexity to improve generalization.

**Definitions**
- Bias: Error due to overly simplistic models that cannot capture the underlying patterns in the data. High bias typically leads to underfitting.
- Variance: Error due to models that are too complex and overly sensitive to the training data. High variance typically leads to overfitting.

Bias-Variance Tradeoff
- The goal is to find a balance between bias and variance to minimize the total error, which includes both these components and the irreducible error (noise inherent in the data). This tradeoff can be visualized as follows:

- High Bias, Low Variance: The model is too simple, leading to systematic errors (underfitting). Predictions are consistent but inaccurate.
- Low Bias, High Variance: The model is too complex, capturing noise along with the underlying data patterns (overfitting). Predictions are accurate on the training data but inconsistent on new data.

Evaluating and Adjusting Variance

Training vs. Validation Error:

- High Bias: Both training and validation errors are high.
- High Variance: Training error is low, but validation error is high.

Model Complexity Adjustment:

- For high bias, increase model complexity (e.g., add more features or use a more complex model).
- For high variance, decrease model complexity (e.g., simplify the model, add regularization).

In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate, train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from GenerateAttributes import GenerateAttributes
from sklearn.metrics import make_scorer, r2_score, mean_absolute_error, mean_squared_error

# Load the dataset
data = pd.read_csv('dataset_full.csv')
data.drop(data[data['ocean_proximity'].isin(['ISLAND'])].index, inplace=True)

# Split the data into features and target variable
X = data.drop('median_house_value', axis=1)
y = data['median_house_value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=X['ocean_proximity'])

# Generate features after splitting to prevent data leakage
X_train['income_to_value_ratio'] = X_train['median_income'] / (y_train + 1)  # +1 to avoid division by zero
X_test['income_to_value_ratio'] = X_test['median_income'] / (y_test + 1)

# Define feature categories
categories = ['ocean_proximity']
num_features = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
                'total_bedrooms', 'population', 'households', 'median_income', 'income_to_value_ratio']
features_to_generate = ['total_rooms', 'households', 'population', 'total_bedrooms','median_icome']

# Define transformers
numerical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ("onehot", OneHotEncoder(sparse_output=False))
])

# Define column transformer
ct1 = ColumnTransformer([
    ("num", numerical_transformer, num_features),
    ("cat", categorical_transformer, categories)
], remainder='passthrough')

ct1.set_output(transform='pandas')
ct1.verbose_feature_names_out = False

# Define preprocessing pipeline
preprocessor = Pipeline([
    ('ct1', ct1),
    ('generate', GenerateAttributes(columns=features_to_generate))
])


# Define base models
base_models = [
    #('rf', RandomForestRegressor(n_estimators=100,max_depth=20,min_samples_split=20,min_samples_leaf=10,random_state=42)),
    #('xgb', XGBRegressor(n_estimators=100, random_state=42)),
    ('gb', GradientBoostingRegressor(learning_rate=0.1, n_estimators=100,min_samples_leaf=5,min_samples_split=5,max_depth=10)),
    ('hr', HistGradientBoostingRegressor(max_iter=100,max_depth=10,learning_rate=0.1,l2_regularization=0.1,max_bins=255,random_state=42))
]

# Define the stacking model
stacking_model = StackingRegressor(
    estimators=base_models,
    final_estimator=Ridge(alpha=1.0)
)


# Define the model pipeline
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', stacking_model)
])

# Define custom scoring functions
def bias_scorer(y_true, y_pred):
    return np.mean(y_pred - y_true)

scoring = {
    'neg_mean_squared_error': 'neg_mean_squared_error',
    'neg_mean_absolute_error': 'neg_mean_absolute_error',
    'r2': 'r2',
    'bias': make_scorer(bias_scorer)
}

# Perform cross-validation
cv_results = cross_validate(model_pipeline, X_train, y_train, cv=5, scoring=scoring, return_train_score=True)

# Extract and convert negative scores to positive
train_mse_scores = -cv_results['train_neg_mean_squared_error']
test_mse_scores = -cv_results['test_neg_mean_squared_error']
train_mae_scores = -cv_results['train_neg_mean_absolute_error']
test_mae_scores = -cv_results['test_neg_mean_absolute_error']
train_r2_scores = cv_results['train_r2']
test_r2_scores = cv_results['test_r2']
train_bias_scores = cv_results['train_bias']
test_bias_scores = cv_results['test_bias']

# Compute mean and variance for each metric
metrics = {
    "Train MSE": train_mse_scores,
    "Test MSE": test_mse_scores,
    "Train MAE": train_mae_scores,
    "Test MAE": test_mae_scores,
    "Train R2": train_r2_scores,
    "Test R2": test_r2_scores,
    "Train Bias": train_bias_scores,
    "Test Bias": test_bias_scores
}

for metric_name, values in metrics.items():
    mean_value = np.mean(values)
    variance_value = np.var(values)
    print(f"{metric_name}:")
    print(f"  Mean: {mean_value}")
    print(f"  Variance: {variance_value}")
    print(f"  Values: {values}\n")
